# Approach
<!-- Describe your approach to solving the problem. -->  
本题基本类似39，但是有两个区别：1.一个元素最多只能抽取一次 2.candidates中有重复元素！！！如果candidates中没有重复元素，为了确保最后的结果中的集合不重复，我们要定义一个startIndex，让每层的递归从i+1开始抽取！但是此时candidates中有重复元素，就算我们每层都从下一个元素i+1开始抽取，最终结果也可能会重复！因此仅仅只定义startIndex不足以确保最终结果中的集合不重复！例如candidates=[10,1,2,7,6,1,5]，target=8，那么如果从1开始我们可以找到[1,2,5]，如果从2开始哪怕我们下一个元素从2的右边(i+1)开始，我们还是可以找到[2,1,5]，重复了！！

去什么重：组合问题可以抽象为树形结构，那么重复的元素在这个树形结构上是有两个维度的，一个维度是同一树枝上(递归，纵向遍历)重复的，一个维度是同一树层（for loop，横向遍历）上重复的。根据题意，元素在同一个组合内是可以重复的，只要candidates中有这些重复元素就行，但两个组合之间不能完全相同！由于同一树枝上的都是一个组合里的元素，不用去重，因此我们要去重的是同一树层上的重复元素。例如candidates=[1,1,2]，target=3，在第一层，先取1，那么这个1下面的第二层还可以选第二个1！同一组合(树枝)内可以重复！这样下去最终会得到[1,2]。如果接下来第一层取第二个1，那么最终还是会得到[1,2]，重复了！综上，我们同一层上不能取重复元素！   

如何实现去重：1.将candidates从小到大排序 2.在每一层中，如果当前元素等于之前的元素，那么当前元素跳过！这就意味着如果每个集合的第一个元素不相同，那么最终集合一定不同！这是因为candidates已经从小到大排序了！因此最终结果里的集合都是递增序列，如果每个集合的第一个元素不相同，由于集合是递增的，这些集合一定不相同！！！

![WechatIMG80.png](attachment:WechatIMG80.png)   

难点在于如何判断是同一层上的重复元素！有两种方式：1.定义一个used数组，used[i]表示candidates[i]是否被添加进path。对于一个candidates[i]来说，如果used[i-1]==true，即candidates[i-1]在path中，就算candidates[i]==candidates[i-1]，candidates[i]也完全可以加进去，因为它们是同一树枝的元素！因此只有used[i-1]==False，即candidates[i-1]和candidates[i]是同一层元素时，如果candidates[i]==candidates[i-1]，那么candidates[i]不能被加进path！2.利用startIndex，当i>startIndex表示同一层元素！


剪枝操作和39完全相同！


# Note
回溯算法模板框架：
void backtracking(参数) {
    if (终止条件) {
        存放结果;
        return;
    }

    for (选择：本层集合中元素（树中节点孩子的数量就是集合的大小）) {
        处理节点;
        backtracking(路径，选择列表); // 递归
        回溯，撤销处理结果
    }
}

# Code

In [ ]:
# 利用used数组对同一层的重复元素去重
class Solution:
    def combinationSum2(self, candidates: List[int], target: int) -> List[List[int]]:
        path = []
        result = []
        # used[i]表示candidates[i]是否在path中
        used = [False] * len(candidates)
        # 从小到大排序
        candidates.sort()
        self.backtracking(candidates, used, target, 0, path, result)
        return result

    def backtracking(self, candidates, used, target, startIndex, path, result):
        if sum(path) > target:
            return
            
        if sum(path) == target:
            result.append(path[:])
            return
        
        for i in range(startIndex, len(candidates)):
            # used[i-1]==False表示candidates[i]和candidates[i-1]是同层的元素
            if not (i > 0 and candidates[i] == candidates[i - 1] and used[i - 1] == False):
                used[i] = True
                path.append(candidates[i])
                self.backtracking(candidates, used, target, i + 1, path, result)
                used[i] = False
                path.pop()


In [ ]:
# 利用startIndex对同一层的重复元素去重
class Solution:
    def combinationSum2(self, candidates: List[int], target: int) -> List[List[int]]:
        path = []
        result = []
        # 从小到大排序
        candidates.sort()
        self.backtracking(candidates, target, 0, path, result)
        return result

    def backtracking(self, candidates, target, startIndex, path, result):

        if sum(path) > target:
            return
            
        if sum(path) == target:
            result.append(path[:])
            return
        
        for i in range(startIndex, len(candidates)):
            # 利用i>startIndex去确定是同一层的元素
            if not (i > startIndex and candidates[i] == candidates[i - 1]):
                path.append(candidates[i])
                self.backtracking(candidates, target, i + 1, path, result)
                path.pop()

In [ ]:
# 剪枝优化
class Solution:
    def combinationSum2(self, candidates: List[int], target: int) -> List[List[int]]:
        path = []
        result = []
        # 本来就要sort
        candidates.sort()
        self.backtracking(candidates, target, 0, path, result)
        return result

    def backtracking(self, candidates, target, startIndex, path, result):
        # 不需要检查sum有没有大于target！
        
        if sum(path) == target:
            result.append(path[:])
            return
        
        for i in range(startIndex, len(candidates)):
            if not (i > startIndex and candidates[i] == candidates[i - 1]):
                # 剪枝优化
                temp = path.copy()
                temp.append(candidates[i])
                if sum(temp) > target:
                    break

                path.append(candidates[i])
                self.backtracking(candidates, target, i + 1, path, result)
                path.pop()

# 剪枝优化，定义sum参数累加求和
class Solution:
    def combinationSum2(self, candidates: List[int], target: int) -> List[List[int]]:
        path = []
        result = []
        candidates.sort()
        self.backtracking(candidates, target, 0, 0, path, result)
        return result

    def backtracking(self, candidates, target, startIndex, sum, path, result):

        if sum == target:
            result.append(path[:])
            return
        
        for i in range(startIndex, len(candidates)):
            if not (i > startIndex and candidates[i] == candidates[i - 1]):
                if sum + candidates[i] > target:
                    break

                sum += candidates[i]
                path.append(candidates[i])
                self.backtracking(candidates, target, i + 1, sum, path, result)
                sum -= candidates[i]
                path.pop()

# 剪枝优化，用continue代替if not
class Solution:
    def combinationSum2(self, candidates: List[int], target: int) -> List[List[int]]:
        path = []
        result = []
        candidates.sort()
        self.backtracking(candidates, target, 0, 0, path, result)
        return result

    def backtracking(self, candidates, target, startIndex, sum, path, result):

        if sum == target:
            result.append(path[:])
            return
        
        for i in range(startIndex, len(candidates)):
            if i > startIndex and candidates[i] == candidates[i - 1]:
                continue

            if sum + candidates[i] > target:
                break

            sum += candidates[i]
            path.append(candidates[i])
            self.backtracking(candidates, target, i + 1, sum, path, result)
            sum -= candidates[i]
            path.pop()